<div align="center">

# **Vigentes**

</div>

## Librerias

In [60]:
source("../src/data/vigentes_diarias.R")

In [61]:
library(readxl)
library(dplyr)

## Data

In [62]:
vigentes <- read_excel("../data/input/Grupo_P11.xlsx")
head(vigentes)

FECINICIO,FECFIN,VLRPRISUSCR,VLRASEGU,VLRASEGURC,PTD,PPD,PH,PPH,RC
<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-01-08,2020-01-08,1379000,46490000,2e+09,1,1,1,1,1
2019-01-08,2020-01-08,1414000,62590000,2e+09,1,1,1,1,1
2019-01-08,2020-01-08,1369000,36490000,2e+09,1,1,1,1,1
2019-01-08,2020-01-08,1379000,41990000,2e+09,1,1,1,1,1
2019-01-08,2020-01-08,1379000,45390000,2e+09,1,1,1,1,1
2019-01-08,2019-09-01,661678,23500000,3e+09,1,1,1,1,1


In [63]:
names(vigentes)

[1] "FECINICIO"   "FECFIN"      "VLRPRISUSCR" "VLRASEGU"    "VLRASEGURC" 
 [6] "PTD"         "PPD"         "PH"          "PPH"         "RC"

In [64]:
vigentes$PTD <- NULL

In [65]:
vigentes <- vigentes %>%
  rename(
    "inicio" = "FECINICIO",                 # Fecha de inicio
    "fin" = "FECFIN",                       # Fecha de fin  
    "prima" = "VLRPRISUSCR",                # Prima
    "valor_asegurado" = "VLRASEGU",         # Valor asegurado general
    "valor_asegurado_rc" = "VLRASEGURC",    # Valor asegurado RC
    "ppd" = "PPD",                          # Pérdida parcial daños
    "pth" = "PH",                           # Pérdida total hurto
    "pph" = "PPH",                          # Pérdida parcial hurto
    "rc" = "RC"                             # Responsabilidad civil
  )

In [66]:
vigentes <- vigentes %>%
  mutate(
    valor_asegurado = valor_asegurado + valor_asegurado_rc  # Suma los valores asegurados
  )

## Nulos y duplicados

In [67]:
sum(is.na(vigentes))

[1] 0

In [68]:
unique(duplicated(vigentes))

[1] FALSE  TRUE

In [69]:
nrow(vigentes)

[1] 300

In [70]:
nrow(unique(vigentes))

[1] 294

Eliminamos los 6 duplicados

In [71]:
vigentes <- unique(vigentes)

## Fechas y duracion

In [72]:
min(vigentes$inicio)
max(vigentes$inicio)

[1] "2019-01-08 UTC"

[1] "2019-01-09 UTC"

In [73]:
min(vigentes$fin)
max(vigentes$fin)

[1] "2019-01-08 UTC"

[1] "2020-12-29 UTC"

In [74]:
vigentes$duracion <- as.numeric(difftime(vigentes$fin, vigentes$inicio, units = "days")) + 1

In [75]:
sort(unique(vigentes$duracion))

[1]   1  18  24  31  38  54  66  81  84  87  89  99 107 108 109 114 117 122 129
[20] 137 150 152 174 200 206 207 213 225 236 237 251 254 255 256 259 266 267 273
[39] 274 298 303 304 305 326 327 341 348 358 366 390 397 399 406 457 500 501 529
[58] 530 613 661 672 680 688 720 722

In [76]:
nrow(vigentes[vigentes$duracion == 366, ])*100/nrow(vigentes)

[1] 50

In [77]:
nrow(vigentes[vigentes$duracion <= 1, ])*100/nrow(vigentes)

[1] 13.26531

Vamos a considerar como una anomalia y vamos a corregirla. Es probable que todas estas polizas que duran un dia en realidad tengan una cobertura de 1 año y fueron mal digitadas.

In [78]:
vigentes$duracion[vigentes$duracion == 1] <- 366

## Primas

In [79]:
sort(unique(vigentes$prima))

[1]       0   37122   39259  121702  195057  219278  236005  256979  272372
 [10]  279076  286282  287705  300168  314402  316368  339744  379433  391748
 [19]  429136  429589  433938  457431  466781  466783  471575  482875  485120
 [28]  499729  567796  577482  578321  616388  630032  632390  661678  682500
 [37]  727829  730864  732946  745310  755491  757442  785777  789460  828116
 [46]  834927  848437  862121  874197  880272  889474  896118  900000  906420
 [55]  909980  912847  923278  930532  957084  959759  962339  963071  972139
 [64]  974786  981449  990807  995059 1001403 1029542 1051786 1066400 1071535
 [73] 1080388 1081279 1087721 1088898 1090181 1090536 1102719 1106547 1107822
 [82] 1119259 1127394 1142531 1158500 1163300 1164470 1173323 1177840 1191750
 [91] 1196318 1208534 1211117 1235483 1243400 1245881 1265701 1273246 1298888
[100] 1313815 1315473 1331775 1341459 1346364 1354076 1369000 1379000 1381828
[109] 1389005 1396592 1406541 1413088 1414000 1453916 1465697 1492759 1494762
[118] 1499600 1510284 1516410 1547879 1550881 1560635 1566046 1587620 1604646
[127] 1607680 1623631 1640105 1657000 1667829 1673923 1674798 1678396 1683046
[136] 1692470 1697000 1718975 1726781 1741526 1787700 1800000 1806736 1814971
[145] 1822000 1837651 1839044 1853090 1910428 1928666 1961676 1973880 1978748
[154] 2005301 2024954 2050169 2063768 2079587 2082141 2093918 2114656 2128389
[163] 2147781 2171700 2187357 2198108 2208417 2242967 2280028 2309474 2329618
[172] 2364792 2397900 2408417 2426064 2476720 2549353 2626665 2744899 2785210
[181] 2789600 2881582 2933847 2936069 3002697 3004569 3071054 3073296 3156273
[190] 3311157 3314976 3477091 3677232 4081287 4148832 4171758 4212917

In [80]:
nrow(vigentes[vigentes$prima <100000,])/nrow(vigentes)

[1] 0.2517007

No vamos a considerar esto como una anomalia pues podrian ser seguros que pagan mensualmente y hasta ahora han pagado la primer cuota

Nos quedamos solo con las 294 polizas tras eliminar duplicados

## Separacion por cobertura

In [81]:
ppd <- vigentes[vigentes$ppd == 1,]
ppd <- ppd %>% select(-ppd, -pth, -pph, -rc)

In [82]:
pth <- vigentes[vigentes$pth == 1,]
pth <- pth %>% select(-ppd, -pth, -pph, -rc)

In [83]:
pph <- vigentes[vigentes$pph == 1,]
pph <- pph %>% select(-ppd, -pth, -pph, -rc)

In [84]:
rc <- vigentes[vigentes$rc == 1,]
rc <- rc %>% select(-ppd, -pth, -pph, -rc)

## Agrupacion por dia

In [85]:
mean(vigentes_diarias(ppd)$polizas_activas)

[1] 117.0374

In [86]:
mean(vigentes_diarias(pph)$polizas_activas)

[1] 117.0374

In [87]:
mean(vigentes_diarias(pth)$polizas_activas)

[1] 117.5457

In [88]:
mean(vigentes_diarias(rc)$polizas_activas)

[1] 89.41967